# Save results to csv file

In [1]:
import glob, os
from obspy import read, Trace, Stream
import numpy as np
import math as M
import pandas as pd

### Get list of the folders

In [2]:
cwd = os.getcwd()

In [3]:
dirs = [ name for name in os.listdir(os.getcwd()) if os.path.isdir(os.path.join(os.getcwd(), name)) ]

### Remove unnecessary folder

In [4]:
dirs.remove('.ipynb_checkpoints')

### Get the list of the files inside in a folder

In [5]:
sacs = glob.glob(dirs[0] + '/*')

# Create empty database

In [6]:
keys = ['starttime','endtime','network','station','channel','PGA','PGA Position','PGV','PGV Position','Mw','Distance']
database = {key: [] for key in keys}

## Make a function to detect the position and the value of the maximum element in an numpy array

In [7]:
def max_pos_finder(data):
  if abs(data.min()) > data.max():
    max_val = max(data.min(), data.max(), key=abs)
    max_pos = np.where(data == max_val)
    max_pos= max_pos[0]
  else:
      max_val = data.max()
      max_pos = np.where(data == max_val)
      max_pos = max_pos[0]     
  return max_val, max_pos[0]

## Read all sac files 

In [8]:
st_org = Stream()
for sac in sacs:
    st_org += read(sac,format='SAC')

In [9]:
for tr in st_org:
    database['starttime'].append(tr.stats.starttime)
    database['endtime'].append(tr.stats.endtime)
    database['network'].append(tr.stats.network)
    database['station'].append(tr.stats.station)
    database['channel'].append(tr.stats.channel)
    database['Mw'].append(tr.stats.sac.mag)
    database['Distance'].append(tr.stats.sac.dist)
    PGA, PGA_pos = max_pos_finder(tr.data)
    database['PGA'].append(PGA)
    database['PGA Position'].append(PGA_pos*tr.stats.delta)
    tr_vel = tr.copy()
    tr_vel.filter('highpass',freq = 0.05)
    tr_vel.detrend("demean")
    tr_vel.detrend("linear")
    tr_vel.integrate(method='cumtrapz')
    tr_vel.detrend("demean")
    tr_vel.detrend("linear")
    PGV, PGV_pos = max_pos_finder(tr_vel.data)
    database['PGV'].append(PGV)
    database['PGV Position'].append(PGV_pos*tr.stats.delta)

In [10]:
df = pd.DataFrame.from_dict(database)
df.to_csv('/home/dertuncay/Obspy_Tutorial/database.csv',',',header=True,index=False)